http://aagenielsen.dk/tablut_summary.php

http://aagenielsen.dk/historical_hnefatafl_rules.php

![Tablut Setup](/Users/peterbitman/Desktop/Projects/tafl/tablut-layout.png)

### Food for Thought:

## Imports

Let's use tablut first as a) recorded by Linnaeus which is a fun wrinkle, and b) it is 9x9 as opposed to 11x11 Copenhagen Hnetatafl

With my initially considered encoding approach, input size is $3 \times \text{dimension}^2 + \text{dimension}$ so:
- 9x9 = 252
- 11x11 = 374

In [106]:

EMPTY = 0
WHITE = 1
BLACK = 2
KING = 3
CASTLE = 40

turn = WHITE

legal_moves = []

board = [EMPTY]*81

board = [EMPTY, EMPTY, EMPTY, BLACK, BLACK, BLACK, EMPTY, EMPTY, EMPTY,
        EMPTY, EMPTY, EMPTY, EMPTY, BLACK, EMPTY, EMPTY, EMPTY, EMPTY,
        EMPTY, EMPTY, EMPTY, EMPTY, WHITE, EMPTY, EMPTY, EMPTY, EMPTY,
        BLACK, EMPTY, EMPTY, EMPTY, WHITE, EMPTY, EMPTY, EMPTY, BLACK,
        BLACK, BLACK, WHITE, WHITE, KING, WHITE, WHITE, BLACK, BLACK,
        BLACK, EMPTY, EMPTY, EMPTY, WHITE, EMPTY, EMPTY, EMPTY, BLACK,
        EMPTY, EMPTY, EMPTY, EMPTY, WHITE, EMPTY, EMPTY, EMPTY, EMPTY,
        EMPTY, EMPTY, EMPTY, EMPTY, BLACK, EMPTY, EMPTY, EMPTY, EMPTY,
        EMPTY, EMPTY, EMPTY, BLACK, BLACK, BLACK, EMPTY, EMPTY, EMPTY]

## Creating Move Output Index:

Total number of possible moves:
Non-Central Row/File Squares:
- 8 non-central rows and files
- 8*8 = 64 non-central squares
- 8+8 = 16 possible moves from each square
- 1024 possible moves
Central Row/File Squares:
- 16 central row/file squares
- 8+3 = 11 possible moves from each square
- 176 possible moves
Castle Square:
- 1 square
- 4x4 = 16 possible moves

Total Unique Moves:
1024+176+16 = __1216__

__Too many possible moves to do manually__

Next task is to create a code snippet that will create this dictionary for me, in a reproducible way

In [155]:
outputIndex = {}

LEN_ROW = 9
NUM_SQUARES = LEN_ROW**2
CASTLE_SQUARE = int((NUM_SQUARES-1)/2)
CENTRE_COL = [(LEN_ROW*x)+(LEN_ROW//2) for x in range(0,9)]
CENTRE_ROW = [x for x in range(int(CASTLE_SQUARE-(LEN_ROW//2)), int(CASTLE_SQUARE+(LEN_ROW//2)+1))]

if CASTLE_SQUARE != int(CASTLE_SQUARE):
    print('invalid dimensions')

index_value = 0

def add_squares(start_square, first_square, last_square, step):
    global index_value
    for end_square in range(first_square, last_square, step):
        if end_square != start_square:
            outputIndex[f"({start_square}, {end_square})"] = index_value
            index_value += 1

for start_square in range(0, NUM_SQUARES):
    # first squares in row/col of considered starting square
    first_square_row = (start_square // LEN_ROW) * 9
    first_square_col = start_square - first_square_row
    # King Square
    if start_square == CASTLE_SQUARE:
        # add row moves
        add_squares(start_square=start_square, first_square=first_square_row, last_square=first_square_row+LEN_ROW, step=1)

        # add col moves
        add_squares(start_square=start_square, first_square=first_square_col, last_square=NUM_SQUARES, step=LEN_ROW)

    elif start_square in CENTRE_COL:
        # add row moves
        add_squares(start_square=start_square, first_square=first_square_row, last_square=first_square_row+LEN_ROW, step=1)
        # add col moves
        first_square_col = int(LEN_ROW//2) if start_square < CASTLE_SQUARE else CASTLE_SQUARE + LEN_ROW
        last_square_col = CASTLE_SQUARE if start_square < CASTLE_SQUARE else NUM_SQUARES
        add_squares(start_square=start_square, first_square=first_square_col, last_square=last_square_col, step=LEN_ROW)

    elif start_square in CENTRE_ROW:
        # add row moves
        first_square_row = first_square_row if start_square < CASTLE_SQUARE else CASTLE_SQUARE+1
        last_square_row = CASTLE_SQUARE if start_square < CASTLE_SQUARE else int(CASTLE_SQUARE + LEN_ROW//2 + 1)
        add_squares(start_square=start_square, first_square=first_square_row, last_square=last_square_row, step=1)
        # add col moves
        add_squares(start_square=start_square, first_square=first_square_col, last_square=NUM_SQUARES, step=LEN_ROW)

    else:
        # add row moves
        add_squares(start_square=start_square, first_square=first_square_row, last_square=first_square_row+LEN_ROW, step=1)
        # add col moves
        add_squares(start_square=start_square, first_square=first_square_col, last_square=NUM_SQUARES, step=LEN_ROW)

In [156]:
len(outputIndex)

1216

## Applying Moves

In [73]:
def applyMove(move):
    from_square = move[0]
    to_square = move[1]
    board[to_square] = board[from_square]
    board[from_square] = EMPTY
    if (turn == WHITE):
        turn = BLACK
    else:
        turn = WHITE
    legal_moves = None

## Generating Moves

In [125]:
legal_moves = None

turn = WHITE
board = [EMPTY]*81
board[38] = KING
board[44] = WHITE

In [126]:
def generate_moves():
    """
    Collates and returns all legal moves for given board state.
    """

    if legal_moves == None:
        moves = []

        def check_square_king(from_square, stop, step):
            """
            Helper function that checks if square is a valid for king. Appends valid tuples to moves list.
            """
            for i in range(from_square+step, stop, step):
                if board[i] == EMPTY:
                    moves.append((from_square, i))
                else:
                    break

        def check_square(from_square, stop, step):
            """
            Helper function that checks if square is a valid for attacker/defender. Appends valid tuples to moves list.
            """
            for i in range(from_square+step, stop, step):
                if board[i] == EMPTY:
                    if i == CASTLE:
                        continue
                    else:
                        moves.append((from_square, i))
                else:
                    break

        # for every square, check to see if there is an ally occupying
        for row_start_sq in range(0, 81, 9):
            for col_start_sq in range(0, 9, 1):
                from_square = row_start_sq + col_start_sq
                # takes care of special cases for King piece. Also accounts for fact that KING != turn as all other pieces do
                if board[from_square] == KING and turn == WHITE:
                    # rows
                    check_square_king(from_square=from_square, stop=from_square+9-col_start_sq, step=1)
                    check_square_king(from_square=from_square, stop=row_start_sq-1, step=-1)
                    # cols
                    check_square_king(from_square=from_square, stop=81, step=9)
                    check_square_king(from_square=from_square, stop=col_start_sq-1, step=-9)

                if board[from_square] == turn:
                    # rows
                    check_square(from_square=from_square, stop=from_square+9-col_start_sq, step=1)
                    check_square(from_square=from_square, stop=row_start_sq-1, step=-1)
                    # cols
                    check_square(from_square=from_square, stop=81, step=9)
                    check_square(from_square=from_square, stop=col_start_sq-1, step=-9)
        return moves

In [127]:
generate_moves()

[(38, 39),
 (38, 40),
 (38, 41),
 (38, 42),
 (38, 43),
 (38, 37),
 (38, 36),
 (38, 47),
 (38, 56),
 (38, 65),
 (38, 74),
 (38, 29),
 (38, 20),
 (38, 11),
 (38, 2),
 (44, 43),
 (44, 42),
 (44, 41),
 (44, 39),
 (44, 53),
 (44, 62),
 (44, 71),
 (44, 80),
 (44, 35),
 (44, 26),
 (44, 17),
 (44, 8)]

In [14]:
nums = []

for i in range(0, 10, 2):
    if i%2==0:
        nums.append(i)
    elif i%3==0:
        break

for i in range(0, 10, 3):
    if i%2==0:
        nums.append(i)
    elif i%3==0:
        break

nums

[0, 2, 4, 6, 8, 0]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 9,
 18,
 27,
 36,
 45,
 54,
 63,
 72,
 8,
 17,
 26,
 35,
 44,
 53,
 62,
 71,
 80]

In [137]:
EDGES = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 17, 18, 26, 27, 36, 44,
             45, 53, 54, 62, 63, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80}
moves = [(38, 39),
 (38, 40),
 (38, 41),
 (38, 42),
 (38, 43),
 (38, 37),
 (38, 36),
 (38, 47),
 (38, 56),
 (38, 65),
 (38, 74),
 (38, 29),
 (38, 20),
 (38, 11),
 (38, 2),
 (44, 43),
 (44, 42),
 (44, 41),
 (44, 39),
 (44, 53),
 (44, 62),
 (44, 71),
 (44, 80),
 (44, 35),
 (44, 26),
 (44, 17),
 (44, 8)]

for move in moves:
    if move[1] in EDGES:
        print(move)

(38, 36)
(38, 74)
(38, 2)
(44, 53)
(44, 62)
(44, 71)
(44, 80)
(44, 26)
(44, 17)
(44, 8)


In [149]:
y = 2
x_set = {1} if y == 1 else {2, 3}

def check_x(x):
    if x in x_set:
        print('hi')
    else:
        print('bye')

check_x(x=0)

bye


In [150]:
for i in [1, 4, 5, 6, 7]:
    print(i+1)

2
5
6
7
8


In [154]:
81*3+9-1

251

In [152]:
from keras.models import Model
from keras.layers import *
import tensorflow as tf

In [157]:
inp = Input((251,))

l1 = Dense(128, activation='relu')(inp)
l2 = Dense(128, activation='relu')(l1)
l3 = Dense(128, activation='relu')(l2)
l4 = Dense(128, activation='relu')(l3)
l5 = Dense(128, activation='relu')(l4)

policyOut = Dense(1215, name='policyHead', activation='softmax')(l5)
valueOut = Dense(1, activation='tanh', name='valueHead')(l5)

bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
model = Model(inp, [policyOut,valueOut])
model.compile(optimizer = 'SGD', loss={'valueHead' : 'mean_squared_error', 'policyHead' : bce})

model.save('random_model.keras')

In [1]:
import time

In [10]:
def costly_comp():
    time.sleep(5)
    return 'that was tough'

class Worker():
    motto = costly_comp()

    def __init__(self):
        self.excited = self.motto + '!'

In [11]:
bob = Worker()

In [14]:
bob.excited

'that was tough!'

In [5]:
jeff = Worker()
jeff.motto

'that was tough'

In [26]:
class Log():
    def __init__(self):
        self.log = [1, 2]

    def update_log(self):
        if len(self.log) < 4:
            self.log.append(0)
        else:
            self.log = self.log[1:]
            self.log.append(1)

l = Log()

In [33]:
l.update_log()
l.log

[1, 1, 1, 1]

In [34]:
len(l.log)

4